## Read in peripheral study recording

In [1]:
# given a filename, generate a pandas dataframe with all recording stuff

# L, R, combined fields

In [7]:
from ibmmpy.ibmm import EyeClassifier
# from ibmm_online import EyeClassifierOnline
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import cm

from parser_utils import read_periph_recording

In [8]:
%matplotlib ipympl 

In [11]:
path_to_recording = "C:\\EGLC\\\carla\\PythonAPI\\examples\\recordings\\sud32.txt"
df1 = read_periph_recording(path_to_recording)

25060it [01:05, 382.54it/s]


## Preliminary analysis

In [ ]:
# plots of hits misses vs ecc

## Gaze event detection and OFD based analysis